In [1]:
from clickhouse_driver import Client
# client = Client(host='192.168.113.129', port='9999', database='movie', user='default') 
client = Client(host='localhost', port='9000', database='movie', user='default') 
client.execute('show tables')  # 测试连接

[('credits',),
 ('keywords',),
 ('keywords_metadata',),
 ('movies_metadata',),
 ('ratings',),
 ('ratings_small',)]

In [2]:
sql = " SELECT \
            movieId, \
            visitParamExtractInt(json, 'id') AS keyId \
        FROM ( \
            SELECT \
                k.id AS movieId, \
                k.keywords AS data, \
                JSONExtractArrayRaw(data) AS arr, \
                arrayJoin(arr) AS json \
            FROM keywords as k\
        ) ORDER BY (movieId, keyId);"

res = client.execute(sql)

In [3]:
i = -1
index = {}
while i != len(res)-1:
    i += 1
    movieId = res[i][0]
    keywords_index = []
    while (i < len(res)-1) and (movieId == res[i+1][0]):
        keywords_index.append(res[i][1])
        i += 1
        movieId = res[i][0]
    keywords_index.append(res[i][1])
    index[movieId] = keywords_index
    # print(keywords_index)

In [4]:
# 检查是否有遗漏
test = client.execute("select id from keywords where keywords != '[]';")
for item in test:
    i = item[0]
    if i not in index: print(i)

In [14]:
# Update
for k in tqdm(index.keys()):
    sql = f"ALTER TABLE keywords UPDATE keywords_index={index[k]} WHERE id={k}"
    # print(sql)
    client.execute(sql)
    # if(k==11): break

100%|██████████| 31087/31087 [2:14:48<00:00,  3.14it/s]


In [18]:
# 1-5.4s
# 5-6.8s
# 31087-14m.23.2s
print(len(index))
from tqdm import tqdm 

cnt = 0
for k in tqdm(index.keys()):
    print(k)
    cnt += 1
    if cnt == 10:break
print(max(index.keys()))
index[468707]

31087


  0%|          | 0/31087 [00:00<?, ?it/s]

2
3
5
6
11
12
13
14
15
16


468707


[2486, 14751, 202882]